In [ ]:

from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签]
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [2]:
import pandas as pd
df=pd.read_csv('Daily_RMSE.csv')
df['date']=pd.to_datetime(df['date'])
df.head()

/tmp/ipykernel_13380/4033203392.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['date']=pd.to_datetime(df['date'])


,date,Doy,Daily_RMSE,GPT2w_ZTD,GAMIT_ZTD,Int,Rand
0,2009-02-11,306,0.16891,-1.671140,0.411879,0.345004,0.650788
1,2009-03-11,307,0.19556,-1.631435,0.418651,0.345004,0.412306
2,2009-04-11,308,0.19142,-1.590069,0.446279,0.345004,0.688613
3,2009-05-11,309,0.19040,-1.547070,0.383870,0.345004,0.031724
4,2009-06-11,310,0.18975,-1.502468,0.437124,0.345004,0.526168


In [3]:
plt.plot(df['date'],df['Daily_RMSE'],marker='o',linestyle='--',color='r')
# 设置X轴日期格式
date_format = mdates.DateFormatter('%Y-%m-%d')  # 设置日期格式
plt.gca().xaxis.set_major_formatter(date_format)

# 自动调整刻度间距以避免重叠
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=10))  # 间隔一个月显示一个刻度

# 可选：旋转刻度标签，以防止它们重叠
plt.xticks(rotation=45)

plt.title('X轴日期格式示例')
plt.xlabel('日期')
plt.ylabel('数值')
plt.grid(True)
plt.tight_layout()
plt.show()


findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei


In [4]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# 创建一个示例的多变量时间序列数据集，长度为 1680，输入特征数为 3，输出为 1
# 这里使用随机数据作为示例
seq_length = 1680
input_size = 3
output_size = 1

# 生成示例数据
import pandas as pd
df=pd.read_csv('Daily_RMSE.csv')
df.head()
column_names=['Doy','GPT2w_ZTD','Daily_RMSE']
Input=df[column_names]
Output=df['GAMIT_ZTD']
data=np.array(Input)
target=np.array(Output)
# data = np.random.randn(seq_length, input_size)
# target = np.random.randn(seq_length, output_size)

# 将数据转换为 PyTorch 张量，并移动到GPU
data_tensor = torch.FloatTensor(data).to('cuda')
target_tensor = torch.FloatTensor(target).to('cuda')

# 定义一个简单的多层感知器 (MLP) 模型
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# 初始化模型并将其移动到GPU
hidden_size = 64  # 隐藏层神经元数量
model = MLP(input_size, hidden_size, output_size).to('cuda')

# 定义损失函数
criterion = nn.MSELoss()

# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 存储每时刻的 MAE 和 MSE
train_mae_values = []
train_mse_values = []

# 存储每时刻的预测 MAE 和 MSE
predicted_mae_values = []
predicted_mse_values = []

# 存储每时刻的偏差（bias）
train_bias_values = []
predicted_bias_values = []

# 存储损失函数值
loss_values = []

# 训练模型
num_epochs = 30
for epoch in range(num_epochs):
    mae_sum = 0.0
    mse_sum = 0.0
    bias_sum = 0.0

    # 前向传播和反向传播
    for i in range(seq_length):
        inputs = data_tensor[i:i+1]
        targets = target_tensor[i:i+1]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # 计算训练结果的 MAE 和 MSE
        mae = torch.abs(outputs - targets).mean().item()
        mse = ((outputs - targets) ** 2).mean().item()

        mae_sum += mae
        mse_sum += mse

        # 计算训练结果的偏差（bias）
        bias = (outputs - targets).mean().item()
        bias_sum += bias

        # 存储每时刻的 MAE 和 MSE
        train_mae_values.append(mae)
        train_mse_values.append(mse)
        train_bias_values.append(bias)

    # 计算平均 MAE 和 MSE
    average_mae = mae_sum / seq_length
    average_mse = mse_sum / seq_length
    average_bias = bias_sum / seq_length

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Average MAE: {average_mae:.4f}, Average MSE: {average_mse:.4f}, Average Bias: {average_bias:.4f}')

    loss_values.append(loss.item())

# 使用模型进行预测并计算预测结果的 MAE 和 MSE
predicted_outputs = []
predicted_mae_sum = 0.0
predicted_mse_sum = 0.0
predicted_bias_sum = 0.0

with torch.no_grad():
    for i in range(seq_length):
        test_input = data_tensor[i:i+1]
        predicted_output = model(test_input)

        # 计算预测结果的 MAE 和 MSE
        predicted_mae = torch.abs(predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_mse = ((predicted_output - target_tensor[i:i+1]) ** 2).mean().item()

        predicted_mae_sum += predicted_mae
        predicted_mse_sum += predicted_mse

        # 计算预测结果的偏差（bias）
        predicted_bias = (predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_bias_sum += predicted_bias

        # 存储每时刻的预测 MAE 和 MSE
        predicted_mae_values.append(predicted_mae)
        predicted_mse_values.append(predicted_mse)
        predicted_bias_values.append(predicted_bias)

    # 计算平均预测 MAE 和 MSE
    average_predicted_mae = predicted_mae_sum / seq_length
    average_predicted_mse = predicted_mse_sum / seq_length
    average_predicted_bias = predicted_bias_sum / seq_length

# 绘制每时刻的 MAE 和 MSE 曲线
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(train_mae_values, label='Train MAE', color='blue')
plt.plot(predicted_mae_values, label='Predicted MAE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE vs Time Step')

plt.subplot(2, 1, 2)
plt.plot(train_mse_values, label='Train MSE', color='blue')
plt.plot(predicted_mse_values, label='Predicted MSE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')

plt.tight_layout()
plt.show()

# 打印和绘制损失函数的曲线
plt.figure(figsize=(8, 4))
plt.plot(range(num_epochs), loss_values, label='Loss', color='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs Epoch')
plt.show()

# 打印训练结果和预测结果的偏差（bias）
print(f'Training Bias: {average_bias:.4f}, Predicted Bias: {average_predicted_bias:.4f}')


In [5]:
plt.plot(train_mae_values[:1680], label='Predicted MSE', color='red')

In [6]:
##Transformer model for time series prediction without error
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Simulated time series data
seq_length = 100
time = np.linspace(0, 10, seq_length)
data = np.sin(time) + np.random.normal(0, 0.1, seq_length)

# Convert data to PyTorch tensor
data_tensor = torch.FloatTensor(data).unsqueeze(1)

# Prepare input and target data
input_seq = data_tensor[:-1]
target_seq = data_tensor[1:]

# Define the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, output_size):
        super(TransformerModel, self).__init__()
        self.fc = nn.Linear(input_size, d_model)
        self.transformer = nn.Transformer(d_model, nhead=2, num_encoder_layers=2)
        self.output = nn.Linear(d_model, output_size)

    def forward(self, src, tgt):
        src = self.fc(src)
        tgt = self.fc(tgt)
        out = self.transformer(src, tgt)
        out = self.output(out)
        return out

# Create the model and define loss and optimizer
input_size = 1
d_model = 64
output_size = 1
model = TransformerModel(input_size, d_model, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 500
losses = []

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(input_seq, input_seq)
    loss = criterion(outputs, target_seq)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

# Prediction
model.eval()
with torch.no_grad():
    predicted = model(input_seq, input_seq)

# Plotting
plt.plot(time[1:], data[1:], label="Actual Data")
plt.plot(time[1:], predicted, label="Predicted Data")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.show()


In [7]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torch

# Set a random seed for reproducibility
torch.manual_seed(0)

# Simulate time series data
seq_length = 1680
input_size = 3
output_size = 1

# Generate example data
df=pd.read_csv('Daily_RMSE.csv')
column_names=['Doy','GPT2w_ZTD','Daily_RMSE']
Input=df[column_names]
Output=df['GAMIT_ZTD']
data=np.array(Input)
target=np.array(Output)

# Convert data to PyTorch tensors
data_tensor = torch.FloatTensor(data)
target_tensor = torch.FloatTensor(target)

# Move data and model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_tensor = data_tensor.to(device)
target_tensor = target_tensor.to(device)

# Define a Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, output_size):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(d_model, nhead=2, num_encoder_layers=2)
        self.fc = nn.Linear(input_size, d_model)
        self.output = nn.Linear(d_model, output_size)

    def forward(self, src, tgt):
        src = self.fc(src)
        tgt = self.fc(tgt)
        out = self.transformer(src, tgt)
        out = self.output(out)
        return out

# Initialize the model and move it to the GPU
d_model = 64  # Transformer's dimension
model = TransformerModel(input_size, d_model, output_size).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Lists to store metrics
mae_values = []
mse_values = []
bias_values = []
loss_values = []

# Training the model
num_epochs = 30
for epoch in range(num_epochs):
    mae_sum = 0.0
    mse_sum = 0.0
    bias_sum = 0.0

    for i in range(seq_length):
        inputs = data_tensor[i:i+1]
        targets = target_tensor[i:i+1]

        optimizer.zero_grad()
        outputs = model(inputs, inputs)  # Use the same input for both src and tgt
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # Calculate MAE and MSE
        mae = torch.abs(outputs - targets).mean().item()
        mse = ((outputs - targets) ** 2).mean().item()

        mae_sum += mae
        mse_sum += mse

        # Calculate bias
        bias = (outputs - targets).mean().item()
        bias_sum += bias

        # Store MAE and MSE at each time step
        mae_values.append(mae)
        mse_values.append(mse)

    # Calculate average MAE and MSE
    average_mae = mae_sum / seq_length
    average_mse = mse_sum / seq_length
    average_bias = bias_sum / seq_length

    loss_values.append(loss.item())

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Average MAE: {average_mae:.4f}, Average MSE: {average_mse:.4f}, Average Bias: {average_bias:.4f}')

# Use the model for prediction and calculate MAE, MSE, and bias
predicted_outputs = []
predicted_mae_sum = 0.0
predicted_mse_sum = 0.0
predicted_bias_sum = 0.0
predicted_mae_concate=[]
predicted_mse_concate=[]
with torch.no_grad():
    for i in range(seq_length):
        test_input = data_tensor[i:i+1]
        predicted_output = model(test_input, test_input)

        # Calculate MAE, MSE, and bias for the predicted outputs
        predicted_mae = torch.abs(predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_mse = ((predicted_output - target_tensor[i:i+1]) ** 2).mean().item()

        predicted_mae_sum += predicted_mae
        predicted_mse_sum += predicted_mse
        predicted_mae_concate.append(predicted_mae)
        predicted_mse_concate.append(predicted_mse)
        predicted_bias = (predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_bias_sum += predicted_bias

# Calculate average MAE, MSE, and bias for the predicted outputs
average_predicted_mae = predicted_mae_sum / seq_length
average_predicted_mse = predicted_mse_sum / seq_length
average_predicted_bias = predicted_bias_sum / seq_length

# Plot MAE and MSE at each time step
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(mae_values, label='MAE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE vs Time Step')

plt.subplot(2, 1, 2)
plt.plot(mse_values, label='MSE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')

plt.tight_layout()
plt.show()

# 打印和绘制损失函数的曲线
plt.figure(figsize=(8, 4))
plt.plot(loss_values, label='Loss', color='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs Epoch')
plt.show()

# 打印训练结果和预测结果的偏差（bias）
print(f'Training Bias: {average_bias:.4f}')
print(f'Predicted Bias: {average_predicted_bias:.4f}')


In [8]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(mae_values[:1680], label='MAE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE vs Time Step')

plt.subplot(2, 1, 2)
plt.plot(mse_values[:1680], label='MSE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')

In [9]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子以便结果可重复
torch.manual_seed(0)

# 模拟时间序列数据
seq_length = 1680
input_size = 3
output_size = 1

# 生成示例数据
# Generate example data
df=pd.read_csv('Daily_RMSE.csv')
column_names=['Doy','GPT2w_ZTD','Daily_RMSE']
Input=df[column_names]
Output=df['GAMIT_ZTD']
data=np.array(Input)
target=np.array(Output)

# 将数据转换为PyTorch张量
data_tensor = torch.FloatTensor(data)
target_tensor = torch.FloatTensor(target)

# 将数据移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_tensor = data_tensor.to(device)
target_tensor = target_tensor.to(device)

# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 使用最后一个时间步的输出
        return out

# 初始化模型
hidden_size = 64
num_layers = 2
model = LSTMModel(input_size, hidden_size, output_size, num_layers).to(device)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 存储每时刻的 MAE 和 MSE
mae_values = []
mse_values = []

# 存储每时刻的偏差（bias）
bias_values = []

# 存储损失函数值
loss_values = []

# 训练模型
num_epochs = 30
for epoch in range(num_epochs):
    mae_sum = 0.0
    mse_sum = 0.0
    bias_sum = 0.0

    for i in range(seq_length):
        inputs = data_tensor[i:i+1].unsqueeze(0)  # 添加unsqueeze(0)以匹配模型的输入维度
        targets = target_tensor[i:i+1]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # 计算 MAE 和 MSE
        mae = torch.abs(outputs - targets).mean().item()
        mse = ((outputs - targets) ** 2).mean().item()

        mae_sum += mae
        mse_sum += mse

        # 计算偏差（bias）
        bias = (outputs - targets).mean().item()
        bias_sum += bias

        # 存储每时刻的 MAE 和 MSE
        mae_values.append(mae)
        mse_values.append(mse)

    # 计算平均 MAE 和 MSE
    average_mae = mae_sum / seq_length
    average_mse = mse_sum / seq_length
    average_bias = bias_sum / seq_length

    loss_values.append(loss.item())

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Average MAE: {average_mae:.4f}, Average MSE: {average_mse:.4f}, Average Bias: {average_bias:.4f}')

# 使用模型进行预测并计算预测结果的 MAE 和 MSE
predicted_outputs = []
predicted_mae_sum = 0.0
predicted_mse_sum = 0.0
predicted_bias_sum = 0.0
predicted_mae_concate_LSTM=[]
predicted_mse_concate_LSTM=[]
with torch.no_grad():
    for i in range(seq_length):
        test_input = data_tensor[i:i+1].unsqueeze(0)  # 添加unsqueeze(0)以匹配模型的输入维度
        predicted_output = model(test_input)

        # 计算预测结果的 MAE 和 MSE
        predicted_mae = torch.abs(predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_mse = ((predicted_output - target_tensor[i:i+1]) ** 2).mean().item()
        predicted_mae_concate_LSTM.append(predicted_mae)
        predicted_mse_concate_LSTM.append(predicted_mse)
        predicted_mae_sum += predicted_mae
        predicted_mse_sum += predicted_mse

        # 计算预测结果的偏差（bias）
        predicted_bias = (predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_bias_sum += predicted_bias

    # 计算平均预测 MAE 和 MSE
    average_predicted_mae = predicted_mae_sum / seq_length
    average_predicted_mse = predicted_mse_sum / seq_length
    average_predicted_bias = predicted_bias_sum / seq_length

# 绘制每时刻的 MAE 和 MSE 曲线
mae_values_LSTM=mae_values
mse_values_LSTM=mse_values
plt.figure(figsize=(12, 6))
plt.subplot(4, 1, 1)
plt.plot(mae_values_LSTM, label='MAE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE vs Time Step')

plt.subplot(4, 1, 2)
plt.plot(mse_values_LSTM, label='MSE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')

plt.subplot(4, 1, 3)
plt.plot(predicted_mae_concate_LSTM, label='MSE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')

plt.subplot(4, 1, 4)
plt.plot(predicted_mse_concate_LSTM, label='MSE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')
plt.tight_layout()
plt.show()

# 打印和绘制损失函数的曲线
plt.figure(figsize=(8, 4))
plt.plot(range(num_epochs), loss_values, label='Loss', color='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs Epoch')
plt.show()

# 打印训练结果和预测结果的偏差（bias）
print(f'Training Bias: {average_bias:.4f}')
print(f'Predicted Bias: {average_predicted_bias:.4f}')


In [10]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子以便结果可重复
torch.manual_seed(0)

# 模拟时间序列数据
seq_length = 1680
input_size = 3
output_size = 1

# 生成示例数据
# 生成示例数据
df=pd.read_csv('Daily_RMSE.csv')
column_names=['Doy','GPT2w_ZTD','Daily_RMSE']
Input=df[column_names]
Output=df['GAMIT_ZTD']
data=np.array(Input)
target=np.array(Output)

# 将数据转换为PyTorch张量
data_tensor = torch.FloatTensor(data)
target_tensor = torch.FloatTensor(target)

# 将数据移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_tensor = data_tensor.to(device)
target_tensor = target_tensor.to(device)

# 定义AutoRegressive Transformer模型
class AutoRegressiveTransformer(nn.Module):
    def __init__(self, input_size, d_model, output_size, num_layers=2, nhead=2):
        super(AutoRegressiveTransformer, self).__init__()
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc = nn.Linear(input_size, d_model)
        self.output = nn.Linear(d_model, output_size)

    def forward(self, x):
        x = self.fc(x)
        out = self.transformer(x, x)  # AutoRegressive
        out = self.output(out)
        return out

# 初始化模型
d_model = 64  # Transformer的维度
num_layers = 2
nhead = 2
model = AutoRegressiveTransformer(input_size, d_model, output_size, num_layers, nhead).to(device)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 存储每时刻的 MAE 和 MSE
mae_values = []
mse_values = []

# 存储每时刻的偏差（bias）
bias_values = []

# 存储损失函数值
loss_values = []

# 训练模型
num_epochs = 30
for epoch in range(num_epochs):
    mae_sum = 0.0
    mse_sum = 0.0
    bias_sum = 0.0

    for i in range(seq_length):
        inputs = data_tensor[i:i+1]
        targets = target_tensor[i:i+1]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # 计算 MAE 和 MSE
        mae = torch.abs(outputs - targets).mean().item()
        mse = ((outputs - targets) ** 2).mean().item()

        mae_sum += mae
        mse_sum += mse

        # 计算偏差（bias）
        bias = (outputs - targets).mean().item()
        bias_sum += bias

        # 存储每时刻的 MAE 和 MSE
        mae_values.append(mae)
        mse_values.append(mse)

    # 计算平均 MAE 和 MSE
    average_mae = mae_sum / seq_length
    average_mse = mse_sum / seq_length
    average_bias = bias_sum / seq_length

    loss_values.append(loss.item())

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Average MAE: {average_mae:.4f}, Average MSE: {average_mse:.4f}, Average Bias: {average_bias:.4f}')

# 使用模型进行预测并计算预测结果的 MAE 和 MSE
predicted_outputs = []
predicted_mae=[]
predicted_mse=[]
predicted_mae_sum = 0.0
predicted_mse_sum = 0.0
predicted_bias_sum = 0.0

with torch.no_grad():
    for i in range(seq_length):
        test_input = data_tensor[i:i+1]
        predicted_output = model(test_input)

        # 计算预测结果的 MAE 和 MSE
        predicted_mae = torch.abs(predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_mse = ((predicted_output - target_tensor[i:i+1]) ** 2).mean().item()

        predicted_mae_sum += predicted_mae
        predicted_mse_sum += predicted_mse
        predicted_mae_concate.append(predicted_mae)
        predicted_mse_concate.append(predicted_mse)
        # 计算预测结果的偏差（bias）
        predicted_bias = (predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_bias_sum += predicted_bias

# 计算平均预测 MAE 和 MSE
average_predicted_mae = predicted_mae_sum / seq_length
average_predicted_mse = predicted_mse_sum / seq_length
average_predicted_bias = predicted_bias_sum / seq_length
print("The average predict mae, mse and bias of AutoRegressiveTransformer are:",average_predicted_mae,average_predicted_mse,average_predicted_bias)

In [11]:

# 绘制每时刻的 MAE 和 MSE 曲线
mae_values_AutoTransformer=mae_values
mse_values_AutoTransformer=mse_values
predicted_mae_concate_AR=predicted_mae_concate
predicted_mse_concate_AR=predicted_mse_concate
plt.figure(figsize=(12, 6))
plt.subplot(4, 1, 1)
plt.plot(mae_values_AutoTransformer, label='MAE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE vs Time Step')

plt.subplot(4, 1, 2)
plt.plot(mse_values_AutoTransformer, label='MSE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')

plt.subplot(4, 1, 3)
plt.plot(predicted_mae_concate_AR, label='MAE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE vs Time Step')

plt.subplot(4, 1, 4)
plt.plot(predicted_mse_concate_AR, label='MSE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')
plt.tight_layout()
plt.show()

# 打印和绘制损失函数的曲线
plt.figure(figsize=(8, 4))
plt.plot(range(num_epochs), loss_values, label='Loss', color='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs Epoch')
plt.show()

# 打印训练结果和预测结果的偏差（bias）
print(f'Training Bias: {average_bias:.4f}')
print(f'Predicted Bias: {average_predicted_bias:.4f}')

In [12]:
import csv

# Create or open a CSV file for writing
with open('your_file.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Example variables
    a=[2,3,4]

    # Write the variables to a CSV file
    csvwriter.writerow([a])

    # You can write more rows of data as needed

# Close the CSV file
csvfile.close()


In [13]:
###Transformer
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torch

# Set a random seed for reproducibility
torch.manual_seed(0)

# Simulate time series data
seq_length = 1680
input_size = 3
output_size = 1

# Generate example data
df=pd.read_csv('Daily_RMSE.csv')
column_names=['Doy','GPT2w_ZTD','Daily_RMSE']
Input=df[column_names]
Output=df['GAMIT_ZTD']
data=np.array(Input)
target=np.array(Output)

# Convert data to PyTorch tensors
data_tensor = torch.FloatTensor(data)
target_tensor = torch.FloatTensor(target)

# Move data and model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_tensor = data_tensor.to(device)
target_tensor = target_tensor.to(device)

# Define a Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, output_size):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(d_model, nhead=2, num_encoder_layers=2)
        self.fc = nn.Linear(input_size, d_model)
        self.output = nn.Linear(d_model, output_size)

    def forward(self, src, tgt):
        src = self.fc(src)
        tgt = self.fc(tgt)
        out = self.transformer(src, tgt)
        out = self.output(out)
        return out

# Initialize the model and move it to the GPU
d_model = 64  # Transformer's dimension
model = TransformerModel(input_size, d_model, output_size).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Lists to store metrics
mae_values = []
mse_values = []
bias_values = []
loss_values = []

# Training the model
num_epochs = 30
for epoch in range(num_epochs):
    mae_sum = 0.0
    mse_sum = 0.0
    bias_sum = 0.0

    for i in range(seq_length):
        inputs = data_tensor[i:i+1]
        targets = target_tensor[i:i+1]

        optimizer.zero_grad()
        outputs = model(inputs, inputs)  # Use the same input for both src and tgt
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # Calculate MAE and MSE
        mae = torch.abs(outputs - targets).mean().item()
        mse = ((outputs - targets) ** 2).mean().item()

        mae_sum += mae
        mse_sum += mse

        # Calculate bias
        bias = (outputs - targets).mean().item()
        bias_sum += bias

        # Store MAE and MSE at each time step
        mae_values.append(mae)
        mse_values.append(mse)

    # Calculate average MAE and MSE
    average_mae = mae_sum / seq_length
    average_mse = mse_sum / seq_length
    average_bias = bias_sum / seq_length

    loss_values.append(loss.item())

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Average MAE: {average_mae:.4f}, Average MSE: {average_mse:.4f}, Average Bias: {average_bias:.4f}')

# Use the model for prediction and calculate MAE, MSE, and bias
predicted_outputs = []
predicted_mae_sum = 0.0
predicted_mse_sum = 0.0
predicted_bias_sum = 0.0
predicted_mae_concate=[]
predicted_mse_concate=[]
with torch.no_grad():
    for i in range(seq_length):
        test_input = data_tensor[i:i+1]
        predicted_output = model(test_input, test_input)

        # Calculate MAE, MSE, and bias for the predicted outputs
        predicted_mae = torch.abs(predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_mse = ((predicted_output - target_tensor[i:i+1]) ** 2).mean().item()

        predicted_mae_sum += predicted_mae
        predicted_mse_sum += predicted_mse
        predicted_mae_concate.append(predicted_mae)
        predicted_mse_concate.append(predicted_mse)
        predicted_bias = (predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_bias_sum += predicted_bias

# Calculate average MAE, MSE, and bias for the predicted outputs
average_predicted_mae = predicted_mae_sum / seq_length
average_predicted_mse = predicted_mse_sum / seq_length
average_predicted_bias = predicted_bias_sum / seq_length

# Plot MAE and MSE at each time step
mae_values_Transformer=[]
mse_values_Transformer=[]
predicted_mae_concate_Transformer=[]
predicted_mse_concate_Transformer=[]
mae_values_Transformer=mae_values
mse_values_Transformer=mse_values
predicted_mae_concate_Transformer=predicted_mae_concate
predicted_mse_concate_Transformer=predicted_mse_concate
plt.figure(figsize=(12, 6))
plt.subplot(4, 1, 1)
plt.plot(mae_values_Transformer, label='MAE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE vs Time Step')

plt.subplot(4, 1, 2)
plt.plot(mse_values_Transformer, label='MSE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')

plt.subplot(4, 1, 3)
plt.plot(predicted_mae_concate_Transformer, label='MAE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE vs Time Step')

plt.subplot(4, 1, 4)
plt.plot(predicted_mse_concate_Transformer, label='MSE', color='red')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')

plt.tight_layout()
plt.show()

# 打印和绘制损失函数的曲线
plt.figure(figsize=(8, 4))
plt.plot(loss_values, label='Loss', color='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs Epoch')
plt.show()

# 打印训练结果和预测结果的偏差（bias）
print(f'Training Bias of Transformer is : {average_bias:.4f}')
print("average_predicted_mae, average_predicted_mse, Predicted Bias of Transformer is :",average_predicted_mae,average_predicted_mse,average_predicted_bias)








import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# 创建一个示例的多变量时间序列数据集，长度为 1680，输入特征数为 3，输出为 1
# 这里使用随机数据作为示例
seq_length = 1680
input_size = 3
output_size = 1

# 生成示例数据
import pandas as pd
df=pd.read_csv('Daily_RMSE.csv')
df.head()
column_names=['Doy','GPT2w_ZTD','Daily_RMSE']
Input=df[column_names]
Output=df['GAMIT_ZTD']
data=np.array(Input)
target=np.array(Output)
# data = np.random.randn(seq_length, input_size)
# target = np.random.randn(seq_length, output_size)

# 将数据转换为 PyTorch 张量，并移动到GPU
data_tensor = torch.FloatTensor(data).to('cuda')
target_tensor = torch.FloatTensor(target).to('cuda')

# 定义一个简单的多层感知器 (MLP) 模型
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# 初始化模型并将其移动到GPU
hidden_size = 64  # 隐藏层神经元数量
model = MLP(input_size, hidden_size, output_size).to('cuda')

# 定义损失函数
criterion = nn.MSELoss()

# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 存储每时刻的 MAE 和 MSE
train_mae_values = []
train_mse_values = []

# 存储每时刻的预测 MAE 和 MSE
predicted_mae_values = []
predicted_mse_values = []

# 存储每时刻的偏差（bias）
train_bias_values = []
predicted_bias_values = []

# 存储损失函数值
loss_values = []

# 训练模型
num_epochs = 30
for epoch in range(num_epochs):
    mae_sum = 0.0
    mse_sum = 0.0
    bias_sum = 0.0

    # 前向传播和反向传播
    for i in range(seq_length):
        inputs = data_tensor[i:i+1]
        targets = target_tensor[i:i+1]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # 计算训练结果的 MAE 和 MSE
        mae = torch.abs(outputs - targets).mean().item()
        mse = ((outputs - targets) ** 2).mean().item()

        mae_sum += mae
        mse_sum += mse

        # 计算训练结果的偏差（bias）
        bias = (outputs - targets).mean().item()
        bias_sum += bias

        # 存储每时刻的 MAE 和 MSE
        train_mae_values.append(mae)
        train_mse_values.append(mse)
        train_bias_values.append(bias)

    # 计算平均 MAE 和 MSE
    average_mae = mae_sum / seq_length
    average_mse = mse_sum / seq_length
    average_bias = bias_sum / seq_length

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Average MAE: {average_mae:.4f}, Average MSE: {average_mse:.4f}, Average Bias: {average_bias:.4f}')

    loss_values.append(loss.item())

# 使用模型进行预测并计算预测结果的 MAE 和 MSE
predicted_outputs = []
predicted_mae_sum = 0.0
predicted_mse_sum = 0.0
predicted_bias_sum = 0.0

with torch.no_grad():
    for i in range(seq_length):
        test_input = data_tensor[i:i+1]
        predicted_output = model(test_input)

        # 计算预测结果的 MAE 和 MSE
        predicted_mae = torch.abs(predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_mse = ((predicted_output - target_tensor[i:i+1]) ** 2).mean().item()

        predicted_mae_sum += predicted_mae
        predicted_mse_sum += predicted_mse

        # 计算预测结果的偏差（bias）
        predicted_bias = (predicted_output - target_tensor[i:i+1]).mean().item()
        predicted_bias_sum += predicted_bias

        # 存储每时刻的预测 MAE 和 MSE
        predicted_mae_values.append(predicted_mae)
        predicted_mse_values.append(predicted_mse)
        predicted_bias_values.append(predicted_bias)

    # 计算平均预测 MAE 和 MSE
    average_predicted_mae = predicted_mae_sum / seq_length
    average_predicted_mse = predicted_mse_sum / seq_length
    average_predicted_bias = predicted_bias_sum / seq_length

# 绘制每时刻的 MAE 和 MSE 曲线
mae_values_BP=[]
mse_values_BP=[]
predicted_mae_values_BP=[]
predicted_mse_values_BP=[]
mae_values_BP=train_mae_values
mse_values_BP=train_mse_values
predicted_mae_values_BP=predicted_mae_values
predicted_mse_values_BP=predicted_mse_values
plt.figure(figsize=(12, 6))
plt.subplot(4, 1, 1)
plt.plot(mae_values_BP, label='Train MAE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE vs Time Step')

plt.subplot(4, 1, 2)
plt.plot(mse_values_BP, label='Train MSE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')

plt.subplot(4, 1, 3)
plt.plot(predicted_mae_values_BP, label='Train MAE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE vs Time Step')

plt.subplot(4, 1, 4)
plt.plot(predicted_mse_values_BP, label='Train MSE', color='blue')
plt.xlabel('Time Step')
plt.ylabel('MSE')
plt.legend()
plt.title('MSE vs Time Step')
plt.tight_layout()
plt.show()

# 打印和绘制损失函数的曲线
plt.figure(figsize=(8, 4))
plt.plot(range(num_epochs), loss_values, label='Loss', color='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs Epoch')
plt.show()

# 打印训练结果和预测结果的偏差（bias）
print(f'Training Bias of BP is : {average_bias:.4f}, Predicted Bias of BP is: {average_predicted_bias:.4f}')
print(f'average_predicted_mae of BP is : {average_predicted_mae:.4f}, average_predicted_mse of BP is: {average_predicted_mse:.4f}')


In [14]:
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']


#plt.plot(mse_values_BP[1580:1680], label='GPT2w-MAE', color='blue')
# sequence = [1, 2, 3]
# multiplier = 2.5  # 浮点数
# result = [x * multiplier for x in sequence]
# print(result)
sequence=mae_values_Transformer[160:260]
multiplier = 0.1
result=[x * multiplier for x in sequence]
sequence=mae_values_Transformer[340:440]
multiplier = 0.1
result1=[x * multiplier for x in sequence]

sequence=mae_values_LSTM[10:110]
multiplier = 0.1
result2=[x * multiplier for x in sequence]
plt.plot(result, label='BP-MAE', color='green')
plt.plot(result1, label='ATT-MAE', color='black')
plt.plot(result2, label='CNN-ATT-MAE', color='red')
plt.xlabel('观测时刻编号', size=14)
plt.ylabel('多测站MAE', size=14)
plt.legend(prop={'size': 12})
#plt.title('拟合精度MAE分布图')
plt.xticks(fontproperties='Times New Roman', size=14)
plt.yticks(fontproperties='Times New Roman', size=14)
plt.savefig('Multi stations_MAE.pdf',format='pdf')
plt.show()


In [15]:
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']


#plt.plot(mse_values_BP[1580:1680], label='GPT2w-MAE', color='blue')

plt.plot(mse_values_Transformer[160:260], label='BP-MSE', color='green')
plt.plot(mse_values_Transformer[340:440], label='ATT-MSE', color='black')
plt.plot(mse_values_LSTM[10:110], label='CNN-ATT-MSE', color='red')
plt.xlabel('观测时刻编号', size=14)
plt.ylabel('多测站MSE', size=14)
plt.legend(prop={'size': 12})
#plt.title('拟合精度MSE分布图')

plt.savefig('Multi stations_MSE.pdf',format='pdf')
plt.show()

In [16]:
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
plt.plot(predicted_mae_values_BP[1200:1300], label='GPT2w-MSE', color='blue')
plt.plot(predicted_mae_concate_LSTM[:100], label='BP-MAE', color='green')
plt.plot(predicted_mae_concate_Transformer[1480:1580], label='ATT-MAE', color='black')
plt.plot(predicted_mae_concate_AR[1580:1680], label='CNN-ATT-MAE', color='red')
plt.xlabel('观测时刻编号')
plt.ylabel('多测站MAE')
plt.legend()
plt.title('预测精度MAE分布图')
plt.show()

In [17]:
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签]
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.plot(predicted_mse_values_BP[1200:1300], label='GPT2w-MSE', color='blue')
plt.plot(predicted_mse_concate_LSTM[:100], label='BP-MSE', color='green')
plt.plot(predicted_mse_concate_Transformer[1480:1580], label='ATT-MSE', color='black')
plt.plot(predicted_mse_concate_AR[1580:1680], label='CNN-ATT-MSE', color='red')
plt.xlabel('观测时刻编号')
plt.ylabel('多测站MSE')
plt.legend()
plt.title('预测精度MSE分布图')
plt.show()

In [18]:
# plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签]
# plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'
plt.plot(mae_values_BP[1580:1680], label='GPT2w-MAE', color='blue')
plt.plot(mae_values_LSTM[:100], label='BP-MAE', color='green')
plt.plot(mae_values_Transformer[1580:1680], label='ATT-MAE', color='black')
plt.plot(predicted_mae_concate_AR[1580:1680], label='CNN-ATT-MAE', color='red')


In [19]:
import matplotlib.pyplot as plt

#测站数目影响
models = ['CNN-ATT', 'ATT','BP','GPT2w']
metrics = ['2', '3', '4', '5']
mae_CNN_ATT = [7.9,7.3,6.79,6.2]
mse_CNN_ATT = [2.1,1.89,1.63,1.56]
mae_ATT = [12.4,11.7,9.65,9.23]
mse_ATT = [3.9,	3.34,	3.06,	2.87]
mae_BP = [13.6,	12.38,	10.75,	8.51]
mse_BP= [6.7	,5.72,	4.38,	4.07]
mae_GPT2w= [14.9,	12.67,	11.23,	9.84]
mse_GPT2w= [30.6,	26.45,	23.86,	20.77]

# plt.plot(metrics, mae_GPT2w, marker='o', color='blue',label='MAE-GPT2w')
# plt.plot(metrics,mae_BP, marker='o',color='green', label='MAE-BP')
# plt.plot(metrics, mae_ATT, marker='o', color='black',label='MAE-ATT')
# plt.plot(metrics,mae_CNN_ATT, marker='o',color='red', label='MAE-CNN_ATT')

plt.plot(metrics, mae_GPT2w,  color='blue',label='MAE-GPT2w')
plt.plot(metrics,mae_BP, color='green', label='MAE-BP')
plt.plot(metrics, mae_ATT,  color='black',label='MAE-ATT')
plt.plot(metrics,mae_CNN_ATT, color='red', label='MAE-CNN_ATT')

plt.xlabel("测站数目",fontsize=14)
plt.ylabel("MAE",fontsize=14)
plt.title("测站数目对预测结果的影响",fontsize=14)

plt.legend(['MAE-GPT2w','MAE-BP', 'MAE-ATT','MAE-CNN_ATT'],loc='upper right',prop={'size': 12})
plt.xticks(fontproperties='Times New Roman', size=14)
plt.yticks(fontproperties='Times New Roman', size=14)
plt.savefig('Num stations_MAE.pdf',format='pdf')
plt.show()

plt.plot(metrics, mse_GPT2w,  color='blue',label='MSE-GPT2w')
plt.plot(metrics,mse_BP,color='green', label='MSE-BP')
plt.plot(metrics, mse_ATT, color='black',label='MSE-ATT')
plt.plot(metrics,mse_CNN_ATT, color='red', label='MSE-CNN_ATT')

# plt.plot(metrics, mse_GPT2w, marker='o', color='blue',label='MSE-GPT2w')
# plt.plot(metrics,mse_BP, marker='o',color='green', label='MSE-BP')
# plt.plot(metrics, mse_ATT, marker='o', color='black',label='MSE-ATT')
# plt.plot(metrics,mse_CNN_ATT, marker='o',color='red', label='MSE-CNN_ATT')
plt.xlabel("测站数目",fontsize=14)
plt.ylabel("MSE",fontsize=14)
plt.title("测站数目对预测结果的影响",fontsize=14)

plt.legend(['MSE-GPT2w','MSE-BP', 'MSE-ATT','MSE-CNN_ATT'],loc='upper right',prop={'size': 12})
plt.xticks(fontproperties='Times New Roman', size=14)
plt.yticks(fontproperties='Times New Roman', size=14)
plt.savefig('Num stations_MSE.pdf',format='pdf')
plt.show()


In [20]:
import matplotlib.pyplot as plt
import numpy as np

# 创建一些示例数据
x = [200,300,100,234,111] # X坐标
y = [332,113,235,333,224]  # Y坐标
sizes = np.random.randint(10, 1000, 5)  # 数据点的大小
colors = np.random.rand(5)  # 数据点的颜色

# 创建一个图表
plt.figure(figsize=(8, 6))

# 绘制不同颜色和大小的圆点
plt.scatter(x, y, s=sizes, c=colors, cmap='jet', alpha=0.7)

# 添加颜色代表的数据范围
cbar = plt.colorbar()
cbar.set_label('数据范围')

# 显示图表
plt.title("不同颜色的圆点表示数据的大小")
plt.xlabel("X轴")
plt.ylabel("Y轴")
plt.grid(True)
plt.show()


In [21]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签]
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
# 创建一些示例数据
x = [22.95633,22.61118,22.84044,22.44682,22.92496] # X坐标
y = [108.63571,108.62662,108.32333,108.27753,108.0437] # Y坐标
sizes =[67.963,
235.135,
82.587,
348.198,
212.574,
] # 数据点的大小
colors = np.random.rand(5)  # 数据点的颜色

# # 创建一个图表
# plt.figure(figsize=(8, 6))
# plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
# plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
# 创建自定义归一化器
norm = Normalize(vmin=min(sizes), vmax=max(sizes))

# 绘制不同颜色和大小的圆点
plt.scatter(x, y, s=sizes, c=colors, cmap='jet', alpha=0.7, edgecolors='k', norm=norm)

# 添加颜色代表的数据范围
cbar = plt.colorbar()
cbar.set_label('数据范围 (大小)')

# 显示图表
plt.title("不同颜色的圆点表示数据的大小")
plt.xlabel("X轴")
plt.ylabel("Y轴")
plt.grid(True)
plt.show()


In [22]:
sizes =[67.963,
235.135,
82.587,
348.198,
212.574,
]
print(min(sizes))

In [23]:
import math
#Calculate ZHD and ZTD
P=1022.15
phi=39.9042
H=43.5
k2_prime=16.52
k3=3.776*100000
Tm=20+273
Rd=287.0464
gm=9.8
e=1.35
ZHD=0.0022768*P/(1-math.cos(2*phi/180*math.pi)-0.00028*H)
print('ZHD',ZHD)
ZWD=0.000001*(k2_prime+k3/(Tm))*(Rd/(gm))*e
print('ZWD',ZWD)
ZTD_GPT2w=ZHD+ZWD
print('ZTD_GPT2w',ZTD_GPT2w)
ZTDsaa=0.0022768*((P+(0.05+1255/Tm)*e)/(1-0.00266*math.cos(2*phi)-0.00028*H))
print('ZTDsaa',ZTDsaa)